# Preparação do ambiente

## Bibliotecas

In [2]:
import pandas as pd
import re
import warnings

from datetime import datetime
from datetime import timedelta

from pathlib import Path

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

## Constantes e funções auxiliares

In [4]:
ISSUES_FOLDER = Path(r'Z:\anatel\inova-fiscaliza\dados-pacp\datasets\inspecao_ecommerce\issues')

# Carga e prepação dos dados

In [33]:
files_issues = [file for file in ISSUES_FOLDER.rglob('*.csv')]

df_issues = pd.concat([pd.read_csv(file,sep=';',quotechar="'") for file in files_issues])
df_issues['Qual foi a data da coleta?'] = pd.to_datetime(df_issues['Qual foi a data da coleta?'],format='mixed')
df_issues['Qual foi a data da coleta?'] = df_issues['Qual foi a data da coleta?'].dt.date

df_issues.head()

,url,Projeto,Tipo,Situação,Título,Descrição,Data de início,Data limite,Atribuído para,Unidade de Lotação,...,Fabricante,Modelo do Produto,Valor,Existe campo código SCH?,Código SCH foi fornecido?,Qual código SCH fornecido?,Código SCH é o do produto?,Código EAN foi fornecido?,Qual código EAN fornecido?,Código EAN é o do produto?
0,https://www.amazon.com.br/Smartphone-INFINIX-C...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000001,"h1. Smartphone INFINIX SMART 7, 3GB RAM 64GB C...",2024-07-08,15/07/2024,werther,GR03,...,Infinix,NaN,719.10,Sim,Sim,5.136232e+10,NaN,Não,NaN,NaN
1,https://www.amazon.com.br/Samsung-Galaxy-A32-I...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000002,"h1. Samsung Galaxy A32 Preto, com Tela Infinit...",2024-07-08,15/07/2024,yoshida,GR03,...,Samsung,NaN,1799.00,Sim,Sim,4.242101e+09,NaN,Não,NaN,NaN
2,https://www.amazon.com.br/Samsung-Bateria-6000...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000003,h1. Samsung Galaxy M34 5G 128GB 6GB RAM Mega B...,2024-07-08,15/07/2024,werther,GR03,...,Samsung,NaN,1149.00,Sim,Sim,9.583230e+10,NaN,Não,NaN,NaN
3,https://www.amazon.com.br/Protetora-Volta-Caso...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000004,h1. Protetora TPU Volta Caso Para Sams Galaxy ...,2024-07-08,15/07/2024,yoshida,GR03,...,F2,NaN,59.00,Não,Não,NaN,NaN,Não,NaN,NaN
4,https://www.amazon.com.br/Smartphone-Xiaomi-Re...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000005,h1. Smartphone Xiaomi Redmi Note 11S 6GB RAM 1...,2024-07-08,15/07/2024,werther,GR03,...,Xiaomi,2201117SY | NOTE 11s | 2201117SY,1228.99,Sim,Sim,2.119221e+10,NaN,Não,NaN,NaN


In [34]:
df_issues_url = df_issues[['Marketplace', 'Qual foi a data da coleta?', 'Título','url']]
df_issues_url = df_issues_url.sort_values(by=['Marketplace', 'Título'])

df_issues_url

,Marketplace,Qual foi a data da coleta?,Título,url
0,Amazon,2024-08-07,INSP_PACP_20240708_000001,https://www.amazon.com.br/Smartphone-INFINIX-C...
1,Amazon,2024-08-07,INSP_PACP_20240708_000002,https://www.amazon.com.br/Samsung-Galaxy-A32-I...
2,Amazon,2024-08-07,INSP_PACP_20240708_000003,https://www.amazon.com.br/Samsung-Bateria-6000...
3,Amazon,2024-08-07,INSP_PACP_20240708_000004,https://www.amazon.com.br/Protetora-Volta-Caso...
4,Amazon,2024-08-07,INSP_PACP_20240708_000005,https://www.amazon.com.br/Smartphone-Xiaomi-Re...
...,...,...,...,...
428,Shopee,2024-07-12,INSP_PACP_20240712_000429,https://shopee.com.br/Samsung-Galaxy-m34-5g-12...
429,Shopee,2024-07-12,INSP_PACP_20240712_000430,https://shopee.com.br/Celular-Xiaomi-Redmi-13c...
430,Shopee,2024-07-12,INSP_PACP_20240712_000431,https://shopee.com.br/Smartphone-Xiaomi-Redmi-...
431,Shopee,2024-07-12,INSP_PACP_20240712_000432,https://shopee.com.br/Xiaomi-Redmi-12C-Celular...


In [35]:
with pd.ExcelWriter('d:\\issues_url.xlsx') as writer:
    for marketplace in df_issues_url['Marketplace'].unique():
        df_issues_url[df_issues_url['Marketplace']==marketplace].to_excel(writer,sheet_name=marketplace,index=False)
        
    

## Resultados da raspagem dos marketplaces

In [16]:
map_columns = {
    'Item': 'item',
    'Texto da Busca' : 'texto_busca',
    'Arquivo da Página' : 'arquivo_pagina',
    'Data da Análise' : 'data_coleta',
    'Data da Coleta' : 'data_coleta',
    'Endereço eletrônico (URL)' : 'url',
    'Descrição do produto no anúncio' : 'descricao_produto',
    'O tipo do produto é de uso proibido no Brasil?' : 'produto_proibido',
    'O produto é passível de homologação na Anatel? (Sim ou Não)' : 'produto_passivel',
    'Tipo do Produto' : 'tipo_produto',
    'Fabricante' : 'fabricante',
    'Modelo do Produto' : 'modelo',
    'Valor do Produto (R$)' : 'valor',
    'Número de Unidades à Venda' : 'unidades_venda',
    'Existe o campo código de homologação no anúncio? (Sim ou Não)' : 'existe_campo_codigo_sch',
    'O código de homologação foi fornecido? (Sim ou Não)' : 'codigo_sch_foi_fornecido',
    'O produto é homologado? (Sim, Não e N.A.)' : 'produto_homologado',
    'Validação do código de homologação - O código de homologação fornecido é o do produto anunciado? (Sim, Não, N.A.)' : 'codigo_sch_corresponde_produto',
    'O código EAN foi fornecido? (Sim ou Não) - Apenas para Smartphones.' : 'codigo_ean_foi_fornecido',
    'O código EAN fornecido corresponde ao produto? (Sim ou Não) -  Apenas para Smartphones.' : 'codigo_ean_corresponde_produto',
    'Observação 1 - Qual Código de Homologação fornecido no anúncio?' : 'qual_codigo_sch_fornecido',
    'Marketplace' : 'marketplace',
    'Página': 'pagina',
    'Arquivo': 'arquivo_pagina',
    'Data' : 'data_coleta',
    'URL' : 'url',
    'Título': 'descricao_produto',
    'Categoria': 'tipo_produto',
    'Modelo': 'modelo',
    'Preço': 'valor',
    'Unidades à Venda': 'unidades_venda',
    'Existe campo código SCH?' : 'existe_campo_codigo_sch',
    'Código SCH foi fornecido?' : 'codigo_sch_foi_fornecido',
    'Código de Homologação': 'qual_codigo_sch_fornecido',
    'Código SCH é o do produto?': 'codigo_sch_corresponde_produto',
    'O produto é homologado?': 'produto_homologado',
    'O código EAN foi fornecido?': 'codigo_ean_foi_fornecido',
    'Código EAN é o do produto?': 'codigo_ean_corresponde_produto',
    'Código EAN-GTIN': 'qual_codigo_ean_fornecido'
}

In [17]:
results_folder = root_path / 'Resultados'
result_files = [file for file in results_folder.rglob('*.xlsx') if actual_collect_date in file.name]

map_marketplace = {
    'amazon': 'Amazon', 
    'ml': 'Mercado Livre',
    'carrefour': 'Carrefour', 
    'magalu': 'Magazine Luiza',
    'shopee': 'Shopee',
    'americanas': 'Americanas',
    'casasbahia': 'Casas Bahia'
}

map_tipo_produto = {
    'Celulares e Smartphones': 'Celulares e Smartphones',
    'smartphone': 'Celulares e Smartphones',
    'Smartphones': 'Celulares e Smartphones'
}

scrape_results = []

for file in result_files:
    
    file_name = file.name
    marketplace_code = file_name.split('_')[0]
    marketplace = map_marketplace[marketplace_code]

    df = pd.read_excel(file,dtype='str')
    # remove os caracteres R$ 
    df['Preço'] = df['Preço'].apply(lambda x: str(x).replace(r'R$','').strip())
    # remove '.' separador de milhares e troca separador de decimais de ',' para '.'
    df['Preço'] = df['Preço'].apply(lambda x: x.replace('.','').replace(',','.') if ',' in x else x)
    
    df['Marketplace'] = marketplace
    
    scrape_results.append(df)

df_produtos = pd.concat(scrape_results)

df_produtos = df_produtos.reset_index(drop=True)
df_produtos.columns = df_produtos.columns.map(map_columns)
df_produtos['tipo_produto'] = df_produtos['tipo_produto'].map(map_tipo_produto)

columns_to_keep = ['tipo_produto', 'texto_busca', 'data_coleta', 'marketplace',
                   'arquivo_pagina', 'url', 'descricao_produto',
                   'fabricante', 'modelo', 'valor', 
                   'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto',
                   'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_produtos = df_produtos[columns_to_keep]

df_produtos = df_produtos.merge(df_homologados,on='qual_codigo_sch_fornecido',how='left')
df_produtos = df_produtos.merge(df_sch_ean,on='qual_codigo_sch_fornecido',how='left')
df_produtos['info_ean_homologados'].fillna('EAN indisponível',inplace=True)
df_produtos['info_modelo_ean'].fillna('EAN indisponível',inplace=True)
df_produtos = df_produtos.fillna('')

valor_pattern = '[0-9]+\.?(?:[0-9]{2})?'
df_produtos['valor'] = df_produtos['valor'].apply(lambda x: x if re.match(valor_pattern,x) else -1).astype(float)

# strip pipe caracter from the end of string
for col in ['info_tipo_produto', 'info_modelos_certificado', 'info_ean_homologados']:
    df_produtos[col] = df_produtos[col].apply(lambda x: x[:-3] if x[-3:]==' | ' else x)

df_produtos

,tipo_produto,texto_busca,data_coleta,marketplace,arquivo_pagina,url,descricao_produto,fabricante,modelo,valor,...,qual_codigo_sch_fornecido,codigo_sch_corresponde_produto,codigo_ean_foi_fornecido,qual_codigo_ean_fornecido,codigo_ean_corresponde_produto,info_fabricante,info_tipo_produto,info_modelos_certificado,info_ean_homologados,info_modelo_ean
0,Celulares e Smartphones,smartphone,2024-07-10 00:00:00,Amazon,amazon_20240710_B0C5Y3BVTH.pdf,https://www.amazon.com.br/Smartphone-Infinix-B...,"Smartphone Infinix Hot 30i, 8GB RAM 128GB, Câm...",Infinix,3901892 | Infinix Hot 30I,998.00,...,051362315156,,Não,,,INFINIX MOBILITY LIMITED,Transceptor de Radiação Restrita | Telefone Mó...,X6515 | X669C | Smart 7 | HOT 30i,7908068807155\n* 7908068807193\n* 790806880828...,X6515: Smart 7\n* X669C: HOT 30i
1,Celulares e Smartphones,smartphone,2024-07-10 00:00:00,Amazon,amazon_20240710_B0BRT5YM3R.pdf,https://www.amazon.com.br/Samsung-Galaxy-Proce...,Samsung Galaxy S23 5G 128GB Tela 6.1'' 8GB RAM...,Samsung,SM-S911BZKJZTO | S23,3297.00,...,171522200953,,Não,,,Samsung Electronics Co Ltd.,Equipamento de Radiocomunicação de Radiação Re...,SM-S911B/DS,7892509126458\n* 7892509126465\n* 789250912647...,SM-S911B/DS: S23 5G (128GB)\n* SM-S911B/DS: S2...
2,Celulares e Smartphones,smartphone,2024-07-10 00:00:00,Amazon,amazon_20240710_B0BVZ65S6Y.pdf,https://www.amazon.com.br/Celular-Xiaomi-Redmi...,Celular Xiaomi Redmi 12C 128GB/4GB de RAM/Dual...,Xiaomi,MZB0DH2EU | Celular Xiaomi Redmi 12C | MZB0DH2EU,772.00,...,217622209185,,Não,,,"Xiaomi Communications Co., Ltd.",Telefone Móvel Celular | Transceptor de Radiaç...,2212ARNC4L | 22120RN86G | 22127PC95G | Redmi 1...,6941812715062\n* 6941812715208\n* 694181271538...,2212ARNC4L: Redmi 12C\n* 22120RN86G: Redmi 12C
3,Celulares e Smartphones,smartphone,2024-07-10 00:00:00,Amazon,amazon_20240710_B0BG361Y98.pdf,https://www.amazon.com.br/BLU-Smartphone-B9-Br...,BLU Smartphone B9 128GB 4GB RAM Bronze 4G Tela...,Blu,B9,749.00,...,130132206887,,Não,,,"BLU PRODUCTS, Inc.",Telefone Móvel Celular | Transceptor de Radiaç...,B9,1220000116917\n* 1220000116924,B9: B9
4,Celulares e Smartphones,smartphone,2024-07-10 00:00:00,Amazon,amazon_20240710_B0BS9Z8GGP.pdf,https://www.amazon.com.br/Samsung-Galaxy-Câmer...,Samsung Galaxy A04s 64GB 4G Wi-Fi Tela 6.5'' D...,Samsung,SM-A047MZWGZTO | Galaxy A32,1139.00,...,077592200953,,Não,,,Samsung Electronics Co Ltd.,Transceptor de Radiação Restrita | Telefone Mó...,SM-A047M/DS,7892509125581\n* 7892509125598\n* 789250912560...,SM-A047M/DS: A04s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,Celulares e Smartphones,smartphone,20240710,Shopee,,https://shopee.com.br/Celular-Xiaomi-Redmi-13C...,Celular Xiaomi Redmi 13C 128GB/4GB RAM,,,819.90,...,,,,,,,,,EAN indisponível,EAN indisponível
436,Celulares e Smartphones,smartphone,20240710,Shopee,,https://shopee.com.br/Jogo-Chave-Combinada-Boc...,Jogo Chave Combinada Boca Estrela 12 Peças Pro...,,,67.98,...,,,,,,,,,EAN indisponível,EAN indisponível
437,Celulares e Smartphones,smartphone,20240710,Shopee,,https://shopee.com.br/Telefone-De-Caixa-Dura-%...,Telefone De Caixa Dura À Prova De Choque Para ...,,,30.50,...,,,,,,,,,EAN indisponível,EAN indisponível
438,Celulares e Smartphones,smartphone,20240710,Shopee,,https://shopee.com.br/Amplificador-De-Imagem-T...,Amplificador De Imagem Tela Telefone Celular A...,,,15.99,...,,,,,,,,,EAN indisponível,EAN indisponível


# Análise

In [23]:
map_fiscais_marketplace_even = {
    'Amazon': {'Atribuído para': 'werther', 'Unidade de Lotação': 'GR03'}, 
    'Mercado Livre': {'Atribuído para': 'costalo', 'Unidade de Lotação': 'UO021'},
    'Carrefour': {'Atribuído para': 'marcosrg', 'Unidade de Lotação': 'UO072'}, 
    'Magazine Luiza': {'Atribuído para': 'caldeira', 'Unidade de Lotação': 'GR04'},
    'Shopee': {'Atribuído para': 'lribeiro', 'Unidade de Lotação': 'GR02'},
    'Americanas': {'Atribuído para': 'humbertovinagre', 'Unidade de Lotação': 'GR01'},
    'Casas Bahia': {'Atribuído para': 'leandrorb', 'Unidade de Lotação': 'GR07'},
}

map_fiscais_marketplace_odd = {
    'Amazon': {'Atribuído para': 'yoshida', 'Unidade de Lotação': 'GR03'}, 
    'Mercado Livre': {'Atribuído para': 'josias', 'Unidade de Lotação': 'UO021'},
    'Carrefour': {'Atribuído para': 'denerreis', 'Unidade de Lotação': 'UO072'}, 
    'Magazine Luiza': {'Atribuído para': 'marcelo', 'Unidade de Lotação': 'GR04'},
    'Shopee': {'Atribuído para': 'danilo', 'Unidade de Lotação': 'GR02'},
    'Americanas': {'Atribuído para': 'beraldi', 'Unidade de Lotação': 'GR01'},
    'Casas Bahia': {'Atribuído para': 'parazi', 'Unidade de Lotação': 'FISF'},
}


data_inicio = datetime.strptime(DATA_COLETA,'%d/%m/%Y')
data_fim = data_inicio + timedelta(days=7)
data_fim = data_fim.strftime('%d/%m/%Y')

df_fiscaliza = df_produtos.copy()

# título da última tarefa no Fiscaliza
# as novas terão numeração sequencial a partir desta
fiscaliza_last_issue = 0
format_title = lambda i: f'INSP_PACP_{actual_collect_date}_{str(i+1+fiscaliza_last_issue).zfill(6)}'

df_fiscaliza['Projeto'] = 'Rotulagem de Dados'

df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)
df_fiscaliza['Descrição'] = df_fiscaliza.apply(lambda row: format_issue(row),axis=1)
df_fiscaliza['Tipo'] = 'Fiscalização de Marketplace'
df_fiscaliza['Situação'] = 'Responder pesquisa'
df_fiscaliza['Data de início'] = data_inicio
df_fiscaliza['Data limite'] = data_fim

# fiscais e unidade de lotação
# df_fiscaliza = df_fiscaliza.join(pd.DataFrame(df_fiscaliza['marketplace'].map(map_fiscais_marketplace).to_list()))
idx = df_fiscaliza.index.to_list()
even_idx = idx[0::2]
odd_idx = idx[1::2]
df_fiscais = pd.DataFrame(df_fiscaliza['marketplace'])
df_fiscais['fiscal_even'] = df_fiscais.iloc[even_idx]['marketplace'].map(map_fiscais_marketplace_even)
df_fiscais['fiscal_odd'] = df_fiscais.iloc[odd_idx]['marketplace'].map(map_fiscais_marketplace_odd)
df_fiscais['fiscal'] = df_fiscais[['fiscal_even','fiscal_odd']].apply(lambda row: row['fiscal_even'] if pd.isna(row['fiscal_odd']) else row['fiscal_odd'],axis=1) 
df_fiscais = pd.DataFrame(df_fiscais['fiscal'].to_list())
df_fiscaliza = df_fiscaliza.join(df_fiscais)

columns_to_issues = ['url', 'Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação', 
                     'tipo_produto', 'texto_busca', 'data_coleta', 'marketplace', 
                     'fabricante', 'modelo', 'valor', 
                     'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto', 
                     'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_fiscaliza = df_fiscaliza[columns_to_issues]

columns_to_rename = ['url', 'Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação',
                     'Qual o tipo do produto? ','Texto da busca','Qual foi a data da coleta?','Marketplace',
                     'Fabricante','Modelo do Produto','Valor',
                     'Existe campo código SCH?','Código SCH foi fornecido?','Qual código SCH fornecido?','Código SCH é o do produto?',
                     'Código EAN foi fornecido?','Qual código EAN fornecido?','Código EAN é o do produto?']
df_fiscaliza.columns = columns_to_rename


# for collect_date in df_fiscaliza['Qual foi a data da coleta?'].unique():
    
#     issues_file_date = datetime.strptime(collect_date,'%d/%m/%Y').strftime('%Y%m%d')
#     issues_output_folder = root_path / 'issues'
#     issues_file_name = issues_output_folder/f'issues_{issues_file_date}.csv'
#     if not issues_file_name.exists():
#         df_fiscaliza[df_fiscaliza['Qual foi a data da coleta?']==collect_date].to_csv(issues_file_name,index=False,sep=";",quotechar="'")
#     else:
#         print(f'Issues file for collect date {collect_date} already exists. Not saved.')

issues_output_folder = root_path / 'issues'
issues_file_name = issues_output_folder/f'issues_{actual_collect_date}.csv'
if not issues_file_name.exists():
    df_fiscaliza[df_fiscaliza['Qual foi a data da coleta?']==collect_date].to_csv(issues_file_name,index=False,sep=";",quotechar="'")
else:
    print(f'Issues file for collect date {collect_date} already exists. Not saved.')

df_fiscaliza

Issues file for collect date 2024-07-10 00:00:00 already exists. Not saved.


,url,Projeto,Tipo,Situação,Título,Descrição,Data de início,Data limite,Atribuído para,Unidade de Lotação,...,Fabricante,Modelo do Produto,Valor,Existe campo código SCH?,Código SCH foi fornecido?,Qual código SCH fornecido?,Código SCH é o do produto?,Código EAN foi fornecido?,Qual código EAN fornecido?,Código EAN é o do produto?
0,https://www.amazon.com.br/Smartphone-Infinix-B...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000001,"h1. Smartphone Infinix Hot 30i, 8GB RAM 128GB,...",2024-07-10,17/07/2024,werther,GR03,...,Infinix,3901892 | Infinix Hot 30I,998.00,Sim,Sim,051362315156,,Não,,
1,https://www.amazon.com.br/Samsung-Galaxy-Proce...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000002,h1. Samsung Galaxy S23 5G 128GB Tela 6.1'' 8GB...,2024-07-10,17/07/2024,yoshida,GR03,...,Samsung,SM-S911BZKJZTO | S23,3297.00,Sim,Sim,171522200953,,Não,,
2,https://www.amazon.com.br/Celular-Xiaomi-Redmi...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000003,h1. Celular Xiaomi Redmi 12C 128GB/4GB de RAM/...,2024-07-10,17/07/2024,werther,GR03,...,Xiaomi,MZB0DH2EU | Celular Xiaomi Redmi 12C | MZB0DH2EU,772.00,Sim,Sim,217622209185,,Não,,
3,https://www.amazon.com.br/BLU-Smartphone-B9-Br...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000004,h1. BLU Smartphone B9 128GB 4GB RAM Bronze 4G ...,2024-07-10,17/07/2024,yoshida,GR03,...,Blu,B9,749.00,Sim,Sim,130132206887,,Não,,
4,https://www.amazon.com.br/Samsung-Galaxy-Câmer...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000005,h1. Samsung Galaxy A04s 64GB 4G Wi-Fi Tela 6.5...,2024-07-10,17/07/2024,werther,GR03,...,Samsung,SM-A047MZWGZTO | Galaxy A32,1139.00,Sim,Sim,077592200953,,Não,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,https://shopee.com.br/Celular-Xiaomi-Redmi-13C...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000436,h1. Celular Xiaomi Redmi 13C 128GB/4GB RAM\n ...,2024-07-10,17/07/2024,danilo,GR02,...,,,819.90,,,,,,,
436,https://shopee.com.br/Jogo-Chave-Combinada-Boc...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000437,h1. Jogo Chave Combinada Boca Estrela 12 Peças...,2024-07-10,17/07/2024,lribeiro,GR02,...,,,67.98,,,,,,,
437,https://shopee.com.br/Telefone-De-Caixa-Dura-%...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000438,h1. Telefone De Caixa Dura À Prova De Choque P...,2024-07-10,17/07/2024,danilo,GR02,...,,,30.50,,,,,,,
438,https://shopee.com.br/Amplificador-De-Imagem-T...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000439,h1. Amplificador De Imagem Tela Telefone Celul...,2024-07-10,17/07/2024,lribeiro,GR02,...,,,15.99,,,,,,,
